<a href="https://colab.research.google.com/github/matsunagalab/mdzen/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDZen: AI-Powered Molecular Dynamics Agent

**Interactive AI assistant for setting up MD simulations with Gradio interface**

This notebook provides a tabbed chat interface to interact with the MDZen AI agent:

## Workflow

**Phase 1: Setup** - Describe your simulation, the agent will:
- Analyze your request and ask clarifying questions
- Fetch structures from PDB/AlphaFold
- Generate a structured `SimulationBrief`

**Phase 2: Execute** - Step-by-step workflow execution:
1. `prepare_complex` - Prepare protein + parameterize ligands (GAFF2/AM1-BCC)
2. `solvate` - Add water box + ions
3. `build_topology` - Generate Amber topology (tleap)
4. `run_simulation` - Execute MD with OpenMM (NPT ensemble)

**Visualization** - Interactive 3D trajectory viewer with py3Dmol

---

## Quick Start

1. **Set API key** in Colab secrets (ANTHROPIC_API_KEY, OPENAI_API_KEY, or GOOGLE_API_KEY)
2. **Run Setup cell** - installs Konda + dependencies (~5-10 min)
3. **Start chatting!** - use the Gradio interface

**Example prompts:**
- "Setup MD for PDB 1AKE in water, 1 ns at 300K"
- "I want to simulate lysozyme (PDB 1LYZ) with explicit solvent"
- "Run a short simulation of insulin (PDB 4INS), chain A only"

---
## Setup: Install Konda and Dependencies

**Konda** is a simple wrapper for conda/mamba in Google Colab.
- No kernel restart needed (unlike condacolab)
- Uses mamba for fast package installation
- Installation takes ~5-10 minutes

**API Key**: Set one of the following in Colab secrets:
- `ANTHROPIC_API_KEY` for Claude
- `OPENAI_API_KEY` for GPT-4
- `GOOGLE_API_KEY` for Gemini

In [ ]:
import sys
import os
import time

IN_COLAB = 'google.colab' in sys.modules

# ============================================================================
# Detect and set API keys from Colab secrets
# ============================================================================
detected_provider = None

if IN_COLAB:
    from google.colab import userdata
    
    # Try to detect API keys from Colab secrets
    api_keys = {
        'ANTHROPIC_API_KEY': 'anthropic',
        'OPENAI_API_KEY': 'openai',
        'GOOGLE_API_KEY': 'google',
    }
    
    for key_name, provider in api_keys.items():
        try:
            key_value = userdata.get(key_name)
            if key_value:
                os.environ[key_name] = key_value
                if detected_provider is None:
                    detected_provider = provider
                print(f"{key_name} loaded from Colab secrets ({provider})")
        except:
            pass
    
    if detected_provider is None:
        print("WARNING: No API key found in Colab secrets!")
        print("Please add one of: ANTHROPIC_API_KEY, OPENAI_API_KEY, or GOOGLE_API_KEY")
        print("Go to: Settings (gear icon) > Secrets")
    else:
        print(f"\nUsing {detected_provider.upper()} as LLM provider")
else:
    # Local - check environment variables
    for key_name in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GOOGLE_API_KEY']:
        if os.environ.get(key_name):
            detected_provider = key_name.split('_')[0].lower()
            print(f"{key_name} found in environment ({detected_provider})")
            break

# ============================================================================
# Install dependencies
# ============================================================================
if IN_COLAB:
    start_time = time.time()

    # Install Konda (no kernel restart needed!)
    print("\nInstalling Konda...")
    !pip install -q konda
    import konda
    konda.install()

    # Accept conda terms of service
    print("\nAccepting conda terms of service...")
    !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main 2>/dev/null || true
    !conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r 2>/dev/null || true

    # Install conda packages via mamba (faster than conda)
    print("\n" + "="*60)
    print("Installing AmberTools + scientific packages via mamba...")
    print("This takes ~5-10 minutes. Please wait.")
    print("="*60)
    !konda run "mamba install -y -c conda-forge ambertools=23 openmm rdkit pdbfixer" 2>&1 | tail -20
    print(f"\nConda packages installed ({time.time() - start_time:.0f}s)")

    # Clone repository
    print("\nCloning mdzen repository...")
    !rm -rf /content/mdzen
    !git clone -q https://github.com/matsunagalab/mdzen.git /content/mdzen
    %cd /content/mdzen

    # Install pip dependencies
    print("\nInstalling Python dependencies...")
    !pip install -q gradio py3Dmol mdtraj nest_asyncio matplotlib
    !pip install -q google-adk google-genai litellm
    !pip install -q -e .

    # Set AMBERHOME (Konda uses /root/miniforge3 by default)
    os.environ["AMBERHOME"] = "/root/miniforge3"

    # Add paths
    sys.path.insert(0, '/content/mdzen/src')
    sys.path.insert(0, '/content/mdzen')

    total_time = time.time() - start_time
    print(f"\n" + "="*60)
    print(f"Setup complete! ({total_time/60:.1f} minutes)")
    print(f"LLM Provider: {detected_provider.upper() if detected_provider else 'NOT SET'}")
    print("="*60)
    print("\nYou can now proceed to the next cell!")

else:
    # Local development - add src to path
    sys.path.insert(0, './src')
    sys.path.insert(0, '.')
    print("Local environment - dependencies should be pre-installed.")

---
## MDZen Gradio Chat Interface

Interact with the AI agent to set up and execute your MD simulation!

**Tabs:**
- **Phase 1: Setup** - Describe your simulation, the agent generates a SimulationBrief
- **Phase 2: Execute** - Run the workflow step-by-step (type 'start' then 'continue')
- **Visualization** - View trajectory animation with py3Dmol

**Example prompts:**
- "Setup MD for PDB 1AKE in water, 1 ns at 300K"
- "I want to simulate lysozyme (PDB 1LYZ) with explicit solvent"

In [ ]:
import gradio as gr
import asyncio
import nest_asyncio
import json
import traceback
from pathlib import Path
from datetime import datetime

nest_asyncio.apply()

# ============================================================================
# Global State
# ============================================================================
app_state = {
    "session_id": None,
    "session_service": None,
    "session_dir": None,
    "simulation_brief": None,
    "current_step_index": 0,
    "workflow_outputs": {},
    "clarification_agent": None,
    "clarification_toolsets": None,
    "setup_agent": None,
    "setup_toolsets": None,
}

# Workflow steps
SETUP_STEPS = ["prepare_complex", "solvate", "build_topology", "run_simulation"]

# ============================================================================
# Session Initialization
# ============================================================================
async def init_session():
    """Create new session for MD workflow"""
    import sys
    IN_COLAB = 'google.colab' in sys.modules

    # Generate job ID
    import random
    import string
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))

    # Create session directory
    if IN_COLAB:
        base_dir = Path("/content/mdzen/outputs")
    else:
        base_dir = Path("./outputs")

    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)

    app_state["session_id"] = f"job_{job_id}"
    app_state["session_dir"] = str(session_dir)

    return session_dir

# ============================================================================
# Phase 1: Clarification Chat
# ============================================================================
def phase1_chat(message, history):
    """Phase 1: Gather requirements and generate SimulationBrief"""
    try:
        loop = asyncio.get_event_loop()

        # Initialize session if needed
        if app_state["session_dir"] is None:
            loop.run_until_complete(init_session())

        # Lazy import agents
        from mdzen.agents.clarification_agent import create_clarification_agent
        from google.adk.runners import Runner
        from google.genai import types
        from mdzen.state.session_manager import (
            create_session_service,
            initialize_session_state,
            get_session_state,
        )

        # Create session service if not exists
        if app_state["session_service"] is None:
            db_path = Path(app_state["session_dir"]) / "session.db"
            app_state["session_service"] = create_session_service(str(db_path), in_memory=False)

            # Initialize session state
            loop.run_until_complete(initialize_session_state(
                session_service=app_state["session_service"],
                app_name="mdzen",
                user_id="default",
                session_id=app_state["session_id"],
                session_dir=app_state["session_dir"],
            ))

        # Create clarification agent if not exists
        if app_state["clarification_agent"] is None:
            agent, toolsets = create_clarification_agent()
            app_state["clarification_agent"] = agent
            app_state["clarification_toolsets"] = toolsets

        # Run agent
        runner = Runner(
            app_name="mdzen",
            agent=app_state["clarification_agent"],
            session_service=app_state["session_service"],
        )

        user_message = types.Content(
            role="user",
            parts=[types.Part(text=message)],
        )

        response_text = ""

        async def run_agent():
            nonlocal response_text
            async for event in runner.run_async(
                user_id="default",
                session_id=app_state["session_id"],
                new_message=user_message,
            ):
                if event.is_final_response() and event.content:
                    # Extract text from content
                    if hasattr(event.content, 'parts'):
                        for part in event.content.parts:
                            if hasattr(part, 'text'):
                                response_text += part.text
                    else:
                        response_text = str(event.content)

        loop.run_until_complete(run_agent())

        # Check for SimulationBrief in state
        state = loop.run_until_complete(get_session_state(
            app_state["session_service"],
            "mdzen",
            "default",
            app_state["session_id"]
        ))

        if state and state.get("simulation_brief"):
            app_state["simulation_brief"] = state["simulation_brief"]
            # Append brief info to response
            brief = app_state["simulation_brief"]
            if isinstance(brief, dict):
                pdb_id = brief.get('pdb_id', 'N/A')
                temp = brief.get('temperature', 300)
                sim_time = brief.get('simulation_time_ns', 1.0)
                response_text += f"\n\n---\n**SimulationBrief Generated!**\n- PDB: {pdb_id}\n- Temperature: {temp}K\n- Simulation Time: {sim_time}ns\n\nGo to **Phase 2: Execute** tab and type 'start' to begin workflow."

        yield response_text if response_text else "Processing..."

    except Exception as e:
        yield f"Error: {e}\n\n{traceback.format_exc()}"

# ============================================================================
# Phase 2: Step-by-step Workflow Execution
# ============================================================================
def phase2_chat(message, history):
    """Phase 2: Execute workflow steps with user confirmation"""
    try:
        # Check if SimulationBrief exists
        if not app_state["simulation_brief"]:
            yield "Please complete Phase 1 first to generate SimulationBrief.\n\nGo to the **Phase 1: Setup** tab and describe your simulation."
            return

        msg_lower = message.lower().strip()

        # Handle 'status' command
        if msg_lower == 'status':
            current = app_state["current_step_index"]
            if current >= len(SETUP_STEPS):
                yield "Workflow complete! Check the **Visualization** tab."
            else:
                completed = SETUP_STEPS[:current]
                remaining = SETUP_STEPS[current:]
                status = f"**Workflow Status**\n\n"
                status += f"Completed: {completed if completed else 'None'}\n"
                status += f"Current: {SETUP_STEPS[current]} [{current+1}/{len(SETUP_STEPS)}]\n"
                status += f"Remaining: {remaining[1:] if len(remaining) > 1 else 'None'}\n"
                status += f"\nType 'continue' to proceed with {SETUP_STEPS[current]}."
                yield status
            return

        # Handle 'start' or 'continue'
        if msg_lower not in ['start', 'continue', 'next', 'run']:
            yield f"Commands:\n- 'start' or 'continue': Execute next step\n- 'status': Show workflow status\n\nCurrent step: {SETUP_STEPS[app_state['current_step_index']]} [{app_state['current_step_index']+1}/{len(SETUP_STEPS)}]"
            return

        current_step = app_state["current_step_index"]

        if current_step >= len(SETUP_STEPS):
            yield "Workflow complete! Check the **Visualization** tab."
            return

        step_name = SETUP_STEPS[current_step]
        yield f"Starting step [{current_step+1}/{len(SETUP_STEPS)}]: **{step_name}**...\n"

        # Execute the step
        brief = app_state["simulation_brief"]
        session_dir = Path(app_state["session_dir"])

        loop = asyncio.get_event_loop()

        if step_name == "prepare_complex":
            result = loop.run_until_complete(execute_prepare_complex(brief, session_dir))
        elif step_name == "solvate":
            result = loop.run_until_complete(execute_solvate(brief, session_dir))
        elif step_name == "build_topology":
            result = loop.run_until_complete(execute_build_topology(brief, session_dir))
        elif step_name == "run_simulation":
            result = loop.run_until_complete(execute_run_simulation(brief, session_dir))
        else:
            result = f"Unknown step: {step_name}"

        app_state["current_step_index"] += 1

        # Build response
        response = f"**Step {step_name} complete!**\n\n{result}\n\n"

        if app_state["current_step_index"] >= len(SETUP_STEPS):
            response += "---\n**Workflow complete!** Check the **Visualization** tab."
        else:
            next_step = SETUP_STEPS[app_state["current_step_index"]]
            response += f"---\nNext step: **{next_step}** [{app_state['current_step_index']+1}/{len(SETUP_STEPS)}]\nType 'continue' to proceed."

        yield response

    except Exception as e:
        yield f"Error in step: {e}\n\n{traceback.format_exc()}"

# ============================================================================
# Workflow Step Implementations
# ============================================================================
async def execute_prepare_complex(brief, session_dir):
    """Step 1: Fetch and prepare structure"""
    import importlib
    import servers.structure_server as structure_module
    importlib.reload(structure_module)

    pdb_id = brief.get('pdb_id')
    if not pdb_id:
        return "Error: No PDB ID specified in SimulationBrief"

    # Fetch structure
    fetch_result = await structure_module.fetch_molecules(
        pdb_id=pdb_id,
        source="pdb",
        prefer_format="pdb",
        output_dir=str(session_dir)
    )

    if not fetch_result["success"]:
        return f"Fetch failed: {fetch_result.get('errors')}"

    structure_file = fetch_result["file_path"]

    # Prepare complex
    complex_result = structure_module.prepare_complex(
        structure_file=structure_file,
        select_chains=brief.get('select_chains'),
        ph=brief.get('ph', 7.4),
        process_proteins=True,
        process_ligands=True,
        run_parameterization=True,
        output_dir=str(session_dir)
    )

    if not complex_result["success"]:
        return f"Prepare failed: {complex_result.get('errors')}"

    app_state["workflow_outputs"]["structure_file"] = structure_file
    app_state["workflow_outputs"]["merged_pdb"] = complex_result["merged_pdb"]
    app_state["workflow_outputs"]["complex_result"] = complex_result

    return f"Fetched: {Path(structure_file).name}\nPrepared: {len(complex_result['proteins'])} protein(s), {len(complex_result['ligands'])} ligand(s)\nOutput: {Path(complex_result['merged_pdb']).name}"

async def execute_solvate(brief, session_dir):
    """Step 2: Solvate structure"""
    import importlib
    import servers.solvation_server as solvation_module
    importlib.reload(solvation_module)

    merged_pdb = app_state["workflow_outputs"].get("merged_pdb")
    if not merged_pdb:
        return "Error: No merged_pdb from prepare_complex step"

    solvate_result = solvation_module.solvate_structure(
        pdb_file=str(Path(merged_pdb).resolve()),
        output_dir=str(session_dir),
        output_name="solvated",
        dist=brief.get('box_padding', 12.0),
        cubic=brief.get('cubic_box', True),
        salt=True,
        saltcon=brief.get('salt_concentration', 0.15)
    )

    if not solvate_result["success"]:
        return f"Solvate failed: {solvate_result.get('errors')}"

    app_state["workflow_outputs"]["solvated_pdb"] = solvate_result["output_file"]
    app_state["workflow_outputs"]["box_dimensions"] = solvate_result.get("box_dimensions")

    stats = solvate_result.get('statistics', {})
    return f"Solvated: {stats.get('total_atoms', '?')} atoms\nWater molecules: {stats.get('water_molecules', '?')}\nOutput: {Path(solvate_result['output_file']).name}"

async def execute_build_topology(brief, session_dir):
    """Step 3: Build Amber topology"""
    import importlib
    import servers.amber_server as amber_module
    importlib.reload(amber_module)

    solvated_pdb = app_state["workflow_outputs"].get("solvated_pdb")
    if not solvated_pdb:
        return "Error: No solvated_pdb from solvate step"

    # Get ligand parameters if any
    ligand_params = []
    complex_result = app_state["workflow_outputs"].get("complex_result", {})
    for lig in complex_result.get("ligands", []):
        if lig.get("success") and lig.get("mol2_file"):
            ligand_params.append({
                "mol2": lig["mol2_file"],
                "frcmod": lig["frcmod_file"],
                "residue_name": lig["ligand_id"][:3].upper()
            })

    amber_result = amber_module.build_amber_system(
        pdb_file=solvated_pdb,
        ligand_params=ligand_params if ligand_params else None,
        box_dimensions=app_state["workflow_outputs"].get("box_dimensions"),
        water_model=brief.get('water_model', 'tip3p'),
        output_name="system",
        output_dir=str(session_dir)
    )

    if not amber_result['success']:
        return f"Amber build failed: {amber_result.get('errors')}"

    app_state["workflow_outputs"]["parm7"] = amber_result['parm7']
    app_state["workflow_outputs"]["rst7"] = amber_result['rst7']

    return f"Topology: {Path(amber_result['parm7']).name}\nCoordinates: {Path(amber_result['rst7']).name}"

async def execute_run_simulation(brief, session_dir):
    """Step 4: Run MD simulation with OpenMM"""
    import openmm as mm
    from openmm import app, unit
    from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, DCDReporter, PDBFile

    parm7_file = app_state["workflow_outputs"].get("parm7")
    rst7_file = app_state["workflow_outputs"].get("rst7")

    if not parm7_file or not rst7_file:
        return "Error: No topology files from build_topology step"

    # Select platform
    platform = None
    platform_name = "CPU"
    for name in ['CUDA', 'OpenCL', 'CPU']:
        try:
            platform = mm.Platform.getPlatformByName(name)
            platform_name = name
            break
        except:
            continue

    if platform is None:
        return "Error: No OpenMM platform available"

    # Load topology
    prmtop = AmberPrmtopFile(parm7_file)
    inpcrd = AmberInpcrdFile(rst7_file)

    # Simulation parameters
    temperature = brief.get('temperature', 300.0) * unit.kelvin
    pressure = (brief.get('pressure_bar') or 1.0) * unit.atmosphere
    timestep = 2.0 * unit.femtoseconds
    sim_time = brief.get('simulation_time_ns', 0.1)

    # Create system
    system = prmtop.createSystem(
        nonbondedMethod=app.PME,
        nonbondedCutoff=10 * unit.angstrom,
        constraints=app.HBonds,
        rigidWater=True
    )
    system.addForce(mm.MonteCarloBarostat(pressure, temperature, 25))

    # Create simulation
    integrator = mm.LangevinMiddleIntegrator(temperature, 1/unit.picosecond, timestep)
    simulation = Simulation(prmtop.topology, system, integrator, platform)
    simulation.context.setPositions(inpcrd.positions)
    if inpcrd.boxVectors:
        simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

    # Minimize and equilibrate
    simulation.minimizeEnergy(maxIterations=500)
    simulation.context.setVelocitiesToTemperature(temperature)

    # Setup output
    md_dir = session_dir / "md_simulation"
    md_dir.mkdir(exist_ok=True)

    dcd_file = md_dir / "trajectory.dcd"
    total_steps = int(sim_time * 1e6 / 2)  # 2 fs timestep
    report_interval = max(100, total_steps // 100)
    simulation.reporters.append(DCDReporter(str(dcd_file), report_interval))

    # Run simulation
    simulation.step(total_steps)

    # Save final state
    final_pdb = md_dir / "final_state.pdb"
    state = simulation.context.getState(getPositions=True)
    with open(final_pdb, 'w') as f:
        PDBFile.writeFile(simulation.topology, state.getPositions(), f)

    # Store outputs
    app_state["workflow_outputs"]["trajectory"] = str(dcd_file)
    app_state["workflow_outputs"]["final_pdb"] = str(final_pdb)
    app_state["workflow_outputs"]["output_dir"] = str(md_dir)

    return f"Platform: {platform_name}\nSimulation time: {sim_time} ns\nTrajectory: {dcd_file.name}\nFinal structure: {final_pdb.name}"

# ============================================================================
# Visualization Function
# ============================================================================
def visualize_trajectory():
    """Create py3Dmol visualization"""
    try:
        import py3Dmol
        import mdtraj as md
        import numpy as np
        import tempfile

        if not app_state["workflow_outputs"] or 'trajectory' not in app_state["workflow_outputs"]:
            return "<p style='color: orange;'>No trajectory available. Complete the workflow first.</p>"

        traj_file = app_state["workflow_outputs"]['trajectory']
        top_file = app_state["workflow_outputs"]['parm7']

        # Load trajectory
        traj = md.load(traj_file, top=top_file)

        # Select protein only
        protein_indices = traj.topology.select('protein')
        if len(protein_indices) == 0:
            return "<p>No protein atoms found in trajectory</p>"

        traj_protein = traj.atom_slice(protein_indices)

        # Sample frames
        max_frames = 20
        if traj_protein.n_frames > max_frames:
            frame_indices = np.linspace(0, traj_protein.n_frames - 1, max_frames, dtype=int)
            traj_viz = traj_protein[frame_indices]
        else:
            traj_viz = traj_protein

        # Write multi-model PDB
        with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
            tmp_path = tmp.name

        with open(tmp_path, 'w') as f:
            for i in range(traj_viz.n_frames):
                frame_tmp = tmp_path + f".frame{i}.pdb"
                traj_viz[i].save_pdb(frame_tmp, force_overwrite=True)
                with open(frame_tmp, 'r') as ff:
                    content = ff.read()
                f.write(f"MODEL     {i + 1}\n")
                for line in content.split('\n'):
                    if not line.startswith('MODEL') and not line.startswith('ENDMDL') and line.strip():
                        f.write(line + '\n')
                f.write("ENDMDL\n")
                Path(frame_tmp).unlink()

        with open(tmp_path, 'r') as f:
            traj_pdb = f.read()
        Path(tmp_path).unlink()

        # Create 3D view
        view = py3Dmol.view(width=800, height=500)
        view.addModelsAsFrames(traj_pdb, 'pdb')
        view.setStyle({'cartoon': {'color': 'spectrum'}})
        view.zoomTo()
        view.animate({'loop': 'forward', 'reps': 0, 'interval': 100})

        # Get HTML
        html = view._make_html()

        info = f"<p><b>Trajectory Animation:</b> {traj_viz.n_frames} frames | Total time: {traj.time[-1]:.1f} ps</p>"
        return info + html

    except Exception as e:
        return f"<p style='color: red;'>Error: {e}</p>"

# ============================================================================
# Build Gradio Interface
# ============================================================================
with gr.Blocks(title="MDZen: AI-Powered MD Agent", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# MDZen: AI-Powered Molecular Dynamics Agent")

    with gr.Tabs():
        with gr.Tab("Phase 1: Setup"):
            gr.Markdown("""Describe your simulation and I'll help you configure it.

            When the SimulationBrief is generated, go to **Phase 2: Execute** tab.""")
            gr.ChatInterface(
                fn=phase1_chat,
                examples=[
                    "Setup MD for PDB 1AKE in water, 1 ns at 300K",
                    "Simulate lysozyme (PDB 1LYZ) with explicit solvent",
                    "Run a short simulation of insulin (PDB 4INS), chain A only",
                ],
                retry_btn=None,
                undo_btn=None,
            )

        with gr.Tab("Phase 2: Execute"):
            gr.Markdown("""Execute the MD workflow step by step:

            1. **prepare_complex** - Fetch structure and parameterize ligands
            2. **solvate** - Add water box and ions
            3. **build_topology** - Generate Amber topology files
            4. **run_simulation** - Run MD with OpenMM

            **Commands:** 'start', 'continue', 'status'""")
            gr.ChatInterface(
                fn=phase2_chat,
                examples=["start", "continue", "status"],
                retry_btn=None,
                undo_btn=None,
            )

        with gr.Tab("Visualization"):
            gr.Markdown("3D visualization of your simulation results")
            viz_btn = gr.Button("Load Trajectory", variant="primary")
            viz_output = gr.HTML()
            viz_btn.click(visualize_trajectory, outputs=viz_output)

# Launch
demo.launch(share=True, debug=True)

---
## Download Results

In [ ]:
import sys
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if app_state["session_dir"]:
    session_dir = Path(app_state["session_dir"])

    if session_dir.exists():
        print(f"Session directory: {session_dir}")
        print("\nGenerated files:")
        for f in sorted(session_dir.rglob('*')):
            if f.is_file():
                rel_path = f.relative_to(session_dir)
                size_kb = f.stat().st_size / 1024
                print(f"  {rel_path} ({size_kb:.1f} KB)")

        if IN_COLAB:
            from google.colab import files
            import shutil

            zip_name = f"{session_dir.name}.zip"
            shutil.make_archive(str(session_dir), 'zip', session_dir)
            print(f"\nDownloading {zip_name}...")
            files.download(f"{session_dir}.zip")
        else:
            print(f"\nFiles are in: {session_dir}")
    else:
        print("Session directory not found.")
else:
    print("No session available. Run the workflow first.")

---

## Next Steps

1. **Longer simulations**: Modify the simulation time in your Phase 1 request
2. **Analysis**: Use MDTraj for RMSD, RMSF, hydrogen bonds, etc.
3. **Different systems**: Try membrane proteins, protein-ligand complexes
4. **Command line**: Use `main.py run` for local development

For more information, see the [GitHub repository](https://github.com/matsunagalab/mdzen).